# Simple model simulating trading of Token between two agents

In [ ]:
import sys
sys.path.append("..")
import pandas as pd
import plotly.express as px
from radcad import Model, Simulation, Experiment
from radcad.engine import Engine, Backend
import random
from cadEVM.connector  import connect_fork, connect_mainnet
from ape import config, networks, accounts, project, chain, Contract

In [ ]:
%env WEB3_ALCHEMY_PROJECT_ID = YOUR ALCHEMY KEY HERE


In [3]:
# to compile contracts
!ape compile

In [4]:
connect_fork()

INFO: Starting 'Hardhat node' process.


<hardhat chain_id=31337>

In [5]:
dev = accounts.test_accounts[0]
reciever_1 = accounts.test_accounts[1]

In [8]:
contract = project.Token.deploy("Test", "TST", 18, 5000000, sender = dev)

	ValueError: Default token list has not been set.
	(Use `--verbosity DEBUG` to see full stack-trace)


INFO: Confirmed 0x2ef5561c8834d1c13f4e92a1c8561a60d45eafdb622190b43b7163adddfe8477 (total fees paid = 521433000000000)
SUCCESS: Contract 'Token' deployed to: 0x274b028b03A250cA03644E6c578D81f019eE1323


In [9]:
chain.provider.get_block("latest")

Block(num_transactions=1, hash=HexBytes('0x748f61717beab17bc831886d413911e5391dbd00c0818fc32e35c123cec20b3c'), number=18551066, parent_hash=HexBytes('0x2571f8126b0287d8eb504d381ac3f66e002657fc41761b0a541d58361a6c9d8d'), size=3194, timestamp=1699734444, gas_limit=30000000, gas_used=521433, base_fee=0, difficulty=0, total_difficulty=58750003716598352816469)

In [12]:
contract.transfer( accounts.test_accounts[1] , 100000, sender = dev)

INFO: Confirmed 0xf074247ad157be6cc296926885a7f8e81a73610a3b855af13c490816d41b6088 (total fees paid = 51634000000000)


<Receipt 0xf074247ad157be6cc296926885a7f8e81a73610a3b855af13c490816d41b6088>

In [13]:
params = {
    'a_agent': dev,
    'b_agent': reciever_1,
}
params

{'a_agent': <TestAccount 0x1e59ce931B4CFea3fe4B875411e280e173cB7A9C>,
 'b_agent': <TestAccount 0xc89D42189f0450C2b2c3c61f58Ec5d628176A1E7>}

In [14]:
initial_state =  {
      
     'a_balance': contract.balanceOf(params['a_agent']),
      'b_balance': contract.balanceOf(params['b_agent'])
}
initial_state

{'a_balance': 4900000, 'b_balance': 100000}

In [15]:
def p_trade(params, substep, state_history, prev_state, **kwargs):
    global contract
   
    if random.choice([True, False]):
        sender_address = params['a_agent']
        recipient_address = params['b_agent']
    else:
        sender_address = params['b_agent']
        recipient_address = params['a_agent']

    try:
        contract.transfer(recipient_address , 500000, sender = sender_address)
    except:
        contract.transfer(recipient_address , 500000, sender = dev)

    sent = "Sent"

    return {'sent_signal': sent}

In [16]:
def s_update_balance_a(params, substep, state_history, prev_state, policy_input, **kwargs):
    balance = contract.balanceOf(params['a_agent'])
    return 'a_balance', balance 

def s_update_balance_b(params, substep, state_history, prev_state, policy_input, **kwargs):
    balance = contract.balanceOf(params['b_agent'])
    return 'b_balance', balance 


In [17]:
state_update_blocks = [
    {
        'policies': {
            'trade': p_trade,
        },
        'variables': {
            'a_balance': s_update_balance_a,
            'b_balance': s_update_balance_b,
        }
    }

]

In [20]:
TIMESTEPS = 10
RUNS = 2
model = Model(initial_state=initial_state, state_update_blocks=state_update_blocks, params=params)
simulation = Simulation(model=model, timesteps=TIMESTEPS, runs=RUNS)
experiment = Experiment(simulation)



# Select the Pathos backend to avoid issues with multiprocessing and Jupyter Notebooks
experiment.engine = Engine(backend=Backend.PATHOS)

result = experiment.run()

In [21]:
df = pd.DataFrame(result)
df

,a_balance,b_balance,simulation,subset,run,substep,timestep
0,4900000,100000,0,0,1,0,0
1,4900000,100000,0,0,1,1,1
2,4400000,600000,0,0,1,1,2
3,4900000,100000,0,0,1,1,3
4,4400000,600000,0,0,1,1,4
5,3900000,1100000,0,0,1,1,5
6,2900000,2100000,0,0,1,1,6
7,2900000,2100000,0,0,1,1,7
8,3900000,1100000,0,0,1,1,8
9,2900000,2100000,0,0,1,1,9


In [ ]:
px.line(df, x='timestep', y= ['a_balance', 
                              'b_balance'], color='run', title= 'Tokens traded between agents over 2 runs')

## Demonstrating use of already deployed contracts and accounts 

In [137]:
dai = Contract('0x6B175474E89094C44Da98b954EedeAC495271d0F')

In [138]:
dai.approve(dev, 100, sender = dev)

INFO: Confirmed 0xc831b63a6bd01b7ede31b8fbb718739eaacb23182f6b2dd110a3b944d59fda06 (total fees paid = 26186000000000)


<Receipt 0xc831b63a6bd01b7ede31b8fbb718739eaacb23182f6b2dd110a3b944d59fda06>

In [139]:
dai._mutable_methods_

{'approve': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.approve,
 'burn': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.burn,
 'deny': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.deny,
 'mint': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.mint,
 'move': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.move,
 'permit': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.permit,
 'pull': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.pull,
 'push': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.push,
 'rely': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.rely,
 'transfer': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.transfer,
 'transferFrom': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.transferFrom}

In [140]:
dai._view_methods_

{'DOMAIN_SEPARATOR': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.DOMAIN_SEPARATOR,
 'PERMIT_TYPEHASH': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.PERMIT_TYPEHASH,
 'allowance': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.allowance,
 'balanceOf': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.balanceOf,
 'decimals': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.decimals,
 'name': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.name,
 'nonces': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.nonces,
 'symbol': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.symbol,
 'totalSupply': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.totalSupply,
 'version': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.version,
 'wards': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.wards}

In [141]:
whale = accounts["0xab5801a7d398351b8be11c439e05c5b3259aec9b"]

In [142]:
dai.balanceOf(whale)

2112188229596500000

In [143]:
dai.transfer(dev, 100000000, sender = whale)

<Receipt 0xcc56d213b8ed077a4d2f05ea736dc9fdbd3635cbd3f5a27007a3c9a46f043640>

In [144]:
dai.balanceOf(dev)

20503500000

In [145]:
params = {
    'a_agent': dev,
    'b_agent': whale,
}
params

{'a_agent': <TestAccount 0x1e59ce931B4CFea3fe4B875411e280e173cB7A9C>,
 'b_agent': <ImpersonatedAccount 0xAb5801a7D398351b8bE11C439e05C5B3259aeC9B>}

In [146]:
initial_state =  {
      
     'a_balance': dai.balanceOf(params['a_agent']),
      'b_balance': dai.balanceOf(params['b_agent'])
}
initial_state

{'a_balance': 20503500000, 'b_balance': 2112188229496500000}

In [147]:
dai.symbol

<Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.symbol

In [148]:
def p_trade(params, substep, state_history, prev_state, **kwargs):
    dai = Contract('0x6B175474E89094C44Da98b954EedeAC495271d0F')
   
    if random.choice([True, False]):
        sender_address = params['a_agent']
        recipient_address = params['b_agent']
    else:
        sender_address = params['b_agent']
        recipient_address = params['a_agent']

    try:
        
        dai.transfer(recipient_address , 5000000000, sender = sender_address)
    except:
        
        dai.transfer(recipient_address , 5000000000, sender = whale)

    sent = "Sent"

    return {'sent_signal': sent}

In [149]:
def s_update_balance_a(params, substep, state_history, prev_state, policy_input, **kwargs):
    dai = Contract('0x6B175474E89094C44Da98b954EedeAC495271d0F')
    balance = dai.balanceOf(params['a_agent'])
    return 'a_balance', balance 

def s_update_balance_b(params, substep, state_history, prev_state, policy_input, **kwargs):
    dai = Contract('0x6B175474E89094C44Da98b954EedeAC495271d0F')
    balance = dai.balanceOf(params['b_agent'])
    return 'b_balance', balance 


In [150]:
state_update_blocks = [
    {
        'policies': {
            'trade': p_trade,
        },
        'variables': {
            'a_balance': s_update_balance_a,
            'b_balance': s_update_balance_b,
        }
    }

]

In [154]:
TIMESTEPS = 10
RUNS = 2
model = Model(initial_state=initial_state, state_update_blocks=state_update_blocks, params=params)
simulation = Simulation(model=model, timesteps=TIMESTEPS, runs=RUNS)
experiment = Experiment(simulation)



# Select the Pathos backend to avoid issues with multiprocessing and Jupyter Notebooks
experiment.engine = Engine(backend=Backend.PATHOS)

result = experiment.run()

INFO: Confirmed 0x9ecf7065a06f168bd736fc30bf5e9b5ccf91e7253a80683e4b26f0a405834db4 (total fees paid = 34682000000000)
INFO: Confirmed 0xe2dad61dd19e503f47a25cb1b358b28f55572cc9274a68d2210208c765efc9c0 (total fees paid = 34682000000000)
INFO: Confirmed 0x518ee73b78c34c77e444e60c1f84588cff190d2bcf44adbc96414067ba586fda (total fees paid = 34682000000000)
INFO: Confirmed 0x1554762bdb4c802ebb256c95fab1f743ed089590efe8775161fff938e4c9d270 (total fees paid = 34682000000000)
INFO: Confirmed 0xd94babc2a11052f12b1d338bffd352024e628fdd3103a9107a63328f23a041ad (total fees paid = 34682000000000)
INFO: Confirmed 0xbfaf460b3726b5ec96f1ddb183f4b1e842deac00b589fdfb9fdf69cf16e7dde2 (total fees paid = 34682000000000)
INFO: Confirmed 0xabfd9c8a7bf3c07cca63e66addf0ae69ae116c7b49d8c2bd1c573b2616216ea3 (total fees paid = 34682000000000)
INFO: Confirmed 0x985e497b95f82ecb74fe95736c21e841050e67e38b15387a2af36c8886f4bc1a (total fees paid = 34682000000000)
INFO: Confirmed 0xb5664d537ad193ad1ad3c98b93f55854e9d365

In [155]:
df2 = pd.DataFrame(result)
df2

,a_balance,b_balance,simulation,subset,run,substep,timestep
0,20503500000,2112188229496500000,0,0,1,0,0
1,5503500000,2112188239496500000,0,0,1,1,1
2,15503500000,2112188229496500000,0,0,1,1,2
3,20503500000,2112188229496500000,0,0,1,1,3
4,20503500000,2112188229496500000,0,0,1,1,4
5,20503500000,2112188229496500000,0,0,1,1,5
6,20503500000,2112188229496500000,0,0,1,1,6
7,20503500000,2112188229496500000,0,0,1,1,7
8,20503500000,2112188229496500000,0,0,1,1,8
9,15503500000,2112188234496500000,0,0,1,1,9
